##### What is this notebook about?
- This notebook shows details of LLM inference using huggingface API. 


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

#model_name = "meta-llama/Llama-3.2-1B-Instruct"
model_name = "openai-community/gpt2"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

print(model)
print(tokenizer)


/home/sbujimal/miniforge3/envs/mlenv2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
GPT2TokenizerFast(name_or_path='openai-community/gpt2

### Using model() directly
- This generates an CausalLMOutputWithCrossAttentions object with many attributes

In [3]:
# Tokenize input
text = "how are you"
inputs = tokenizer(text, return_tensors="pt").to(model.device)
print(f"input: {inputs}")
 
# Generate output
with torch.no_grad():
    outputs_obj = model(**inputs)
print("outputs_obj type:", type(outputs_obj)) # object transformers.modeling_outputs.CausalLMOutputWithCrossAttentions

# Get output logits
logits = outputs_obj.logits
print("logits shape:", logits.shape) # batchsize, numtokens, vocabsize

# Last token logit
last_token_logits = logits[0, -1, :]

# Generate greedy next token
next_token_id = last_token_logits.argmax()
next_token = tokenizer.decode(next_token_id)
print(f"Greedy: next_token_id: {next_token_id}, next_token: {next_token}")

# Generate topk next tokens
k = 5
topk = torch.topk(last_token_logits, k=k)
next_token_ids = topk.indices
next_tokens = [tokenizer.decode(next_token_id) for next_token_id in next_token_ids]
print(f"Topk: k: {k}, next_token_ids: {next_token_ids}, next_tokens: {next_tokens}")

input: {'input_ids': tensor([[4919,  389,  345]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}
outputs_obj type: <class 'transformers.modeling_outputs.CausalLMOutputWithCrossAttentions'>
logits shape: torch.Size([1, 3, 50257])
Greedy: next_token_id: 1016, next_token:  going
Topk: k: 5, next_token_ids: tensor([1016, 1804, 1701,   30, 4203], device='cuda:0'), next_tokens: [' going', ' doing', '?"', '?', ' feeling']


### Using model.generate() function - return Tensor
- By default, this generates Tensor of next tokens

In [4]:
# Function to tokenize & generate output
def tokenize_generate_response(tokenizer, model, generation_config, text, apply_chat_template=False):

    # Tokenize input
    if apply_chat_template:
        text = tokenizer.apply_chat_template(
            text,
            tokenize=False,
            add_generation_prompt=True
        )
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    print(f"inputs: {inputs}")

    # Output a tensor directly
    outputs = model.generate(
        **inputs,
        generation_config=generation_config
    )
    print("outputs type:", type(outputs)) # Tensor
    print(f"outputs.shape: {outputs.shape}, outputs: {outputs}")  # batchsize, totalnumtokens
    print()

    # Use tokenizer.decode to decode 1 sample at a time
    decode_arg = outputs[0]  # (totalnumtokens, )
    full_response = tokenizer.decode(decode_arg, skip_special_tokens=True) #[0]
    print(f"Full response (including input) using tokenizer.decode({decode_arg}) with input shape: {decode_arg.shape}:")
    print(full_response)
    decode_arg = outputs[0][len(inputs.input_ids[0]):]
    response = tokenizer.decode(decode_arg, skip_special_tokens=True) #[0]
    print(f"Only new tokens response using tokenizer.decode({decode_arg}) with input shape: {decode_arg.shape}:")
    print(response)
    print()

    # Use tokenizer.batch_decode to decode many samples at a time
    decode_arg = outputs
    full_response = tokenizer.batch_decode(outputs, skip_special_tokens=True) #[0]
    print(f"Full response (including input) using tokenizer.batch_decode({decode_arg}) with input shape: {decode_arg.shape}:")
    print(full_response)
    decode_arg = torch.tensor([output[len(input_ids):] for input_ids, output in zip(inputs.input_ids, outputs)]).to(model.device) # Select only new token ixs
    response = tokenizer.batch_decode(decode_arg, skip_special_tokens=True) #[0]
    print(f"Only new tokens response using tokenizer.batch_decode({decode_arg}) with input shape: {decode_arg.shape}:")
    print(response)
    print()

# Generation config
generation_config = GenerationConfig(
    #max_length=256,
    max_new_tokens=1,
    temperature=0.05,
    do_sample=True,
    #do_sample=False,
    use_cache=True,
    skip_special_tokens=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
print(generation_config)
for attr in ["max_new_tokens", "output_scores", "return_dict_in_generate"]:
    attr_name = f"generation_config.{attr}"
    att_val = eval(attr_name)
    print(f"{attr_name}: {att_val}")
print()

# Input
text = "how are you"
tokenize_generate_response(tokenizer, model, generation_config, text)

GenerationConfig {
  "do_sample": true,
  "eos_token_id": 50256,
  "max_new_tokens": 1,
  "pad_token_id": 50256,
  "skip_special_tokens": true,
  "temperature": 0.05
}

generation_config.max_new_tokens: 1
generation_config.output_scores: False
generation_config.return_dict_in_generate: False

inputs: {'input_ids': tensor([[4919,  389,  345]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}
outputs type: <class 'torch.Tensor'>
outputs.shape: torch.Size([1, 4]), outputs: tensor([[4919,  389,  345, 1804]], device='cuda:0')

Full response (including input) using tokenizer.decode(tensor([4919,  389,  345, 1804], device='cuda:0')) with input shape: torch.Size([4]):
how are you doing
Only new tokens response using tokenizer.decode(tensor([1804], device='cuda:0')) with input shape: torch.Size([1]):
 doing

Full response (including input) using tokenizer.batch_decode(tensor([[4919,  389,  345, 1804]], device='cuda:0')) with input shape: torch.Size([1, 4]):
['how are yo

### Using model.generate() function - return scores, etc. 
- if generation_config.return_dict_in_generate = False, it generates GenerateDecoderOnlyOutput object with attributes
- if generation_config.output_scores = True, it generate scores attribute

In [5]:
import torch.nn.functional as F

# Function to tokenize & generate output
def tokenize_generate_response(tokenizer, model, generation_config, text, apply_chat_template=False):

    # Tokenize input
    if apply_chat_template:
        text = tokenizer.apply_chat_template(
            text,
            tokenize=False,
            add_generation_prompt=True
        )
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    print(f"inputs: {inputs}")
    print()

    # Output an object with more information, like scores 
    print("Generating object with scores ...\n")
    generation_config.output_scores= True
    generation_config.return_dict_in_generate=True
    outputs_obj = model.generate(
        **inputs,
        generation_config=generation_config
    )
    print("outputs_obj type:", type(outputs_obj)) # transformers.generation.utils.GenerateDecoderOnlyOutput
    scores = outputs_obj.scores
    sequences = outputs_obj.sequences
    print(f"sequences.shape: {sequences.shape}") # batchsize, totalnumtokens
    print(f"sequences[0].shape: {sequences[0].shape}") # totalnumtokens,
    print(f"scores tuple len: {len(scores)}")  # totalnumtokens
    print(f"scores[0].shape: {scores[0].shape}")  # batchsize, vocabsize
    print()

    # Get scores
    scores_tensor = torch.stack(scores, dim=1) # batchsize, totalnumtokens, vocabsize
    probs = F.softmax(scores_tensor, dim=-1) # batchsize, totalnumtokens, vocabsize
    max_probs_obj = torch.max(probs, dim=-1) # (values, indices) with sizes (batchsize, totalnumtokens) 
    max_probs = max_probs_obj[0] # batchsize, totalnumtokens
    print(f"scores_tensor.shape: {scores_tensor.shape}")
    print(f"probs.shape: {probs.shape}, probs:{probs}")
    print(f"max_probs_obj: {max_probs_obj}")
    print(f"max_probs.shape: {max_probs.shape}, max_probs: {max_probs}")
    print()

    print("Decoding from generate object outputs ... \n")

    # Use tokenizer.decode to decode 1 sample at a time
    decode_arg = sequences[0]  # (totalnumtokens, )
    full_response = tokenizer.decode(decode_arg, skip_special_tokens=True) #[0]
    print(f"Full response (including input) using tokenizer.decode({decode_arg}) with input shape: {decode_arg.shape}:")
    print(full_response)
    decode_arg = sequences[0][len(inputs.input_ids[0]):]
    response = tokenizer.decode(decode_arg, skip_special_tokens=True) #[0]
    print(f"Only new tokens response using tokenizer.decode({decode_arg}) with input shape: {decode_arg.shape}:")
    print(response)
    print()

    # Use tokenizer.batch_decode to decode many samples at a time
    decode_arg = sequences
    full_response = tokenizer.batch_decode(sequences, skip_special_tokens=True) #[0]
    print(f"Full response (including input) using tokenizer.batch_decode({decode_arg}) with input shape: {decode_arg.shape}:")
    print(full_response)
    decode_arg = torch.tensor([output[len(input_ids):] for input_ids, output in zip(inputs.input_ids, sequences)]).to(model.device) # Select only new token ixs
    response = tokenizer.batch_decode(decode_arg, skip_special_tokens=True) #[0]
    print(f"Only new tokens response using tokenizer.batch_decode({decode_arg}) with input shape: {decode_arg.shape}:")
    print(response)
    print()

# Generation config
generation_config = GenerationConfig(
    #max_length=256,
    max_new_tokens=1,
    temperature=0.05,
    do_sample=True,
    #do_sample=False,
    use_cache=True,
    skip_special_tokens=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
print(generation_config)
for attr in ["max_new_tokens", "output_scores", "return_dict_in_generate"]:
    attr_name = f"generation_config.{attr}"
    att_val = eval(attr_name)
    print(f"{attr_name}: {att_val}")
print()

# Input
text = "how are you"
tokenize_generate_response(tokenizer, model, generation_config, text)

GenerationConfig {
  "do_sample": true,
  "eos_token_id": 50256,
  "max_new_tokens": 1,
  "pad_token_id": 50256,
  "skip_special_tokens": true,
  "temperature": 0.05
}

generation_config.max_new_tokens: 1
generation_config.output_scores: False
generation_config.return_dict_in_generate: False

inputs: {'input_ids': tensor([[4919,  389,  345]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}

Generating object with scores ...

outputs_obj type: <class 'transformers.generation.utils.GenerateDecoderOnlyOutput'>
sequences.shape: torch.Size([1, 4])
sequences[0].shape: torch.Size([4])
scores tuple len: 1
scores[0].shape: torch.Size([1, 50257])

scores_tensor.shape: torch.Size([1, 1, 50257])
probs.shape: torch.Size([1, 1, 50257]), probs:tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')
max_probs_obj: torch.return_types.max(
values=tensor([[0.9719]], device='cuda:0'),
indices=tensor([[1016]], device='cuda:0'))
max_probs.shape: torch.Size([1, 1]), max_probs: t